# Federated Isolation Forest  with Flex

In this notebook we show how to use  Isolation Forest model using FLEX library.

First we do all the imports needed.

In [ ]:
from flexanomalies.utils import IsolationForest
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.primitives_iforest import (
    build_server_model_if,
    copy_model_to_clients_if,
    train_if,
    aggregate_if,
    set_aggregated_weights_if,
    get_clients_weights_if,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

 Load data, define model parameters and define model

In [2]:
model_params = {"n_estimators": 100, "contamination": 0.1}

In [3]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [4]:
model = IsolationForest(**model_params)

Once the data  is loaded, we have to federate it. For this we use the FLEX library. There are two ways to federate the data, using an IID distribution or a non IID distribution. For the IID distribution we can use the ìid_distribution function of 

FedDataDistribution. If we use a non-IID distribution, it is necessary to use a custom configuration, such as the one used in the federate_data function. For more information, go to the FLEX library workbooks, and take a look at the Federate Data with 

FLEXible notebooks.

## Creating the federated architecture

When creating the federated architecture, we use FlexPool. Since we are running a client-server architecture, we use the client_server_architecture function. 

In [5]:
flex_dataset = federate_data(5, X_train, y_train)

In [6]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="iforest_server",
    init_func=build_server_model_if,
    model=model,
)

## Run the federated learning experiment and Evaluate

Now, we can run the federated experiment and  evaluate it at the server level.

In [ ]:
pool.servers.map(copy_model_to_clients_if, pool.clients)
pool.clients.map(train_if)
pool.aggregators.map(get_clients_weights_if, pool.clients)
pool.aggregators.map(aggregate_if)
pool.aggregators.map(set_aggregated_weights_if, pool.servers)
output_model = pool.servers._models["iforest_server"]["model"]
output_model.evaluate(X_test, y_test)

In [ ]:
output_model.result_metrics_

In [8]:
save_experiments_results(
    "Iforest",
    output_model,
    "test_iforest_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)

## End of Notebook